# F1 FIA Regulations Debate: A 3-Way Adversarial Conversation

This notebook extends the adversarial chatbot conversation from `day1.ipynb` into a **3-way debate** between Formula 1 entities arguing about FIA Technical Regulations and suspected foul play in car design.

**Setting:** The 2024/2025 F1 season, before the major 2026 regulation overhaul.

**Participants:**

| Role | Model | Provider |
|------|-------|----------|
| **FIA** (mediator, stickler to rules) | `anthropic/claude-3.5-haiku` | OpenRouter |
| **Red Bull Racing** (accuser, wants rival disqualified) | `gpt-4.1-mini` | OpenRouter |
| **McLaren** (defender, argues car meets pre-2026 regs) | `x-ai/grok-4.1-fast` | OpenRouter |

**Scenario:** Red Bull Racing has filed a formal protest with the FIA, alleging that McLaren's car features an aerodynamic device that violates the current FIA Technical Regulations. The FIA must mediate the dispute while both teams argue their case. 5 rounds of heated debate ensue.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set - this is required for all three models")

In [ ]:
openrouter = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

fia_model = "anthropic/claude-3.5-haiku"
redbull_model = "gpt-4.1-mini"
mclaren_model = "x-ai/grok-4.1-fast"

In [ ]:
fia_system = """You are the FIA Race Director and Chief Technical Steward presiding over a formal \
technical hearing during the 2024/2025 Formula 1 season, before the 2026 regulation changes. \
You are a strict, impartial mediator who lives and breathes the FIA Technical Regulations. \
You cite specific articles from the FIA Formula One Technical Regulations (e.g., Article 3.5 on \
bodywork, Article 3.6 on aerodynamic influence). You demand hard evidence from both parties before \
making any ruling. You issue formal warnings when teams become too heated. You keep the proceedings \
orderly and focused on the technical merits. You address the teams formally. Keep responses concise \
and authoritative, no more than 150 words."""

redbull_system = """You are the Team Principal of Red Bull Racing during the 2024/2025 Formula 1 \
season. You have filed a formal protest with the FIA alleging that McLaren's car (MCL38/MCL60) \
features a rear wing element and floor edge design that violates the current FIA Technical \
Regulations before the 2026 rule changes. You are aggressive, confrontational, and relentless. \
You reference specific regulation articles, demand immediate disqualification and points deduction, \
and accuse McLaren of deliberately exploiting loopholes. You cite your own team's compliance as \
evidence of unfair advantage. You are suspicious and believe the FIA is being too lenient. Keep \
responses passionate and forceful, no more than 150 words."""

mclaren_system = """You are the Team Principal of McLaren Racing during the 2024/2025 Formula 1 \
season. Red Bull has accused your car (MCL38/MCL60) of violating FIA Technical Regulations regarding \
its rear wing and floor design. You are diplomatic but firm. You defend your car as fully compliant \
with the current pre-2026 regulations. You cite specific FIA scrutineering approvals your car has \
passed, reference engineering rationale and wind tunnel data, and point out that the FIA's own \
technical delegates inspected and approved the design. You suggest Red Bull's protest is motivated \
by their declining performance rather than genuine regulatory concern. You remain professional but \
don't back down. Keep responses measured and confident, no more than 150 words."""

In [ ]:
def call_fia(conversation):
    messages = [
        {"role": "system", "content": fia_system},
        {"role": "user", "content": f"Here is the conversation so far in this FIA technical hearing:\n\n{conversation}\n\nNow respond as the FIA Race Director. Keep the hearing on track and address the points raised."}
    ]
    response = openrouter.chat.completions.create(model=fia_model, messages=messages)
    return response.choices[0].message.content


def call_redbull(conversation):
    messages = [
        {"role": "system", "content": redbull_system},
        {"role": "user", "content": f"Here is the conversation so far in this FIA technical hearing:\n\n{conversation}\n\nNow respond as the Red Bull Racing Team Principal. Press your case aggressively."}
    ]
    response = openrouter.chat.completions.create(model=redbull_model, messages=messages)
    return response.choices[0].message.content


def call_mclaren(conversation):
    messages = [
        {"role": "system", "content": mclaren_system},
        {"role": "user", "content": f"Here is the conversation so far in this FIA technical hearing:\n\n{conversation}\n\nNow respond as the McLaren Racing Team Principal. Defend your position."}
    ]
    response = openrouter.chat.completions.create(model=mclaren_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
conversation = """\
[HEARING BRIEF — CONFIDENTIAL]
The FIA has convened this extraordinary technical hearing following a formal protest \
lodged by Red Bull Racing (Protest No. FIA-2024-TR-047) against McLaren Racing.

Red Bull Racing alleges that the McLaren MCL60, as raced during the 2024 season and \
carried forward into early 2025 development (MCL60B), incorporates the following \
suspected violations of the FIA Formula One Technical Regulations:

1. REAR WING — A multi-element rear wing assembly whose secondary flap profile allegedly \
exceeds the maximum chord length permitted under Article 3.6.2, generating illegal \
downwash effects beyond the regulated aerodynamic envelope.

2. FLOOR EDGE — A floor edge wing design along the venturi tunnel entrance that Red Bull \
claims deviates from the flat-floor profile mandated by Article 12.2.3, producing \
additional ground-effect downforce outside permitted tolerances.

3. FLEXI-WING CONCERNS — Red Bull further alleges that McLaren's rear wing assembly \
deflects beyond the 10mm limit under the 60N load test specified in Article 3.6.8, \
effectively acting as a drag reduction device at high speed.

Red Bull Racing demands immediate disqualification of McLaren's results from affected \
races, retrospective points deduction, and a full FIA technical audit of the MCL60 \
aerodynamic package. McLaren Racing denies all allegations and asserts that the MCL60 \
has passed every FIA scrutineering check at all events during the 2024/2025 season.

The hearing shall now commence. FIA Race Director to open proceedings.
"""

for i in range(5):
    display(Markdown(f"---\n## Round {i + 1}\n"))

    fia_response = call_fia(conversation)
    conversation += f"FIA: {fia_response}\n\n"
    display(Markdown(f"### 🏁 FIA\n{fia_response}\n"))

    redbull_response = call_redbull(conversation)
    conversation += f"Red Bull Racing: {redbull_response}\n\n"
    display(Markdown(f"### 🐂 Red Bull Racing\n{redbull_response}\n"))

    mclaren_response = call_mclaren(conversation)
    conversation += f"McLaren Racing: {mclaren_response}\n\n"
    display(Markdown(f"### 🟠 McLaren Racing\n{mclaren_response}\n"))